In [1]:
from mvodolagin_personal_imports import *

load_dotenv()

True

In [2]:
from mvodolagin_personal_imports.langchain_stuff import *

In [3]:
stores_df = pd.read_pickle(r"E:\Work\Personal\repos\TextTailor\resources\furniture_stores_example.pickle")
stores_df["url_root"] = stores_df["website"].apply(lambda x: x.split("://", 1)[-1].split("/", 1)[0])

In [4]:
locations_count = stores_df["name"].value_counts()
locations_count_by_url_root = stores_df.groupby("url_root").size()
big_names = locations_count[locations_count >= 5].index
big_urls = locations_count_by_url_root[locations_count_by_url_root >= 5].index
big_chains = stores_df[(stores_df["name"].isin(big_names)) | (stores_df["url_root"].isin(big_urls))]
small_stores = stores_df[(~stores_df["name"].isin(big_names)) & (~stores_df["url_root"].isin(big_urls))]

In [5]:
small_stores.describe()

,rating,user_rating_count
count,340.000000,340.000000
mean,4.460882,615.470588
std,0.385235,1593.118776
min,2.800000,1.000000
25%,4.300000,45.000000
50%,4.500000,137.000000
75%,4.700000,534.250000
max,5.000000,17089.000000


In [6]:
data_dir = Path("./root_pages")

In [19]:
all_dirs = [fp for fp in data_dir.glob("*") if fp.is_dir()]
random.shuffle(all_dirs)

for dir_path in all_dirs:
    url_root = dir_path.name
    print(url_root, len(os.listdir(dir_path)))
    break


www.simsfurnitureco.com 15


In [20]:
patterns = json.load((dir_path / "patterns.json").open(encoding="utf-8"))
pprint(patterns)
category_pattern = re.compile(patterns["category"])
item_pattern = re.compile(patterns["item"])

{'category': '/collections/.*',
 'item': '/collections/[a-z0-9-]+/products/[a-z0-9-]+'}


In [23]:
# item_pattern = re.compile(r"/products/")

In [24]:
from trafilatura import extract

item_html_texts = []

for fp in dir_path.glob("*.html"):
    # print(fp)
    html_text = fp.read_text(encoding="utf-8")
    data = extract(html_text, output_format="json")
    data = json.loads(data)
    partial_url = "/" + data["source"].split("://", 1)[-1].split("/", 1)[-1]
    print(data["source"], bool(category_pattern.match(partial_url)), bool(item_pattern.match(partial_url)))
    if bool(item_pattern.match(partial_url)):
        item_html_texts.append(html_text)
    # break

https://www.simsfurnitureco.com/collections/buffet True False
https://www.simsfurnitureco.com/products/galliden-dining-buffet False True
https://www.simsfurnitureco.com/products/maylee-dining-buffet-and-hutch False True
https://www.simsfurnitureco.com/collections/coaster-z2-premium True False
https://www.simsfurnitureco.com/products/yvette-grey-driftwood-l-shaped-office-desk False True
https://www.simsfurnitureco.com/products/zamora-rectangular-3-door-accent-cabinet-natural False True
https://www.simsfurnitureco.com/collections/outdoor-seating-set True False
https://www.simsfurnitureco.com/products/fynnegan-outdoor-loveseat-with-table-set-of-2 False True
https://www.simsfurnitureco.com/products/fynnegan-outdoor-seating-package False True
https://www.simsfurnitureco.com/products/grasson-lane-outdoor-sofa-and-loveseat-with-ottoman False True
https://www.simsfurnitureco.com/products/mandarin-cape-outdoor-table-and-chairs-set-of-3 False True
https://www.simsfurnitureco.com/products/oasis-c

In [83]:
# significant_text_blocks, labels_to_tags = extract_significant_text_blocks(html_text, min_text_length=100)
# significant_text_blocks = postfilter_results(significant_text_blocks)

In [25]:
len(item_html_texts)

10

In [76]:
from bs4 import BeautifulSoup, Tag


# def get_full_tag_label(tag):
#     # return f"{tag.name}{'#' + tag.get('id') if tag.get('id') else ''}{'.' + tag.get('class')[0] if tag
#     attributes = []
#     if tag.get('id'):
#         attributes.append(f"id='{tag['id']}'")
#     if tag.get('class'):
#         attributes.append(f"class='{' '.join(tag['class'])}'")
# 
#     attr_string = " ".join(attributes)
#     full_label = f"<{tag.name} {attr_string}>".strip()
#     return full_label


def determine_tag_location(tag):
    # Helper function to create a CSS selector for a given tag
    def generate_css_selector(tag):
        selector = tag.name
        if tag.attrs:
            if 'id' in tag.attrs:
                selector += f"#{tag['id']}"
            elif 'class' in tag.attrs:
                # Join all classes into one selector
                classes = '.'.join(tag['class'])
                selector += f".{classes}"
        return selector

    # Helper function to create an XPath for a given tag
    def generate_xpath(tag):
        path = []
        current = tag
        while current.parent and isinstance(current.parent, Tag):
            siblings = [t for t in current.parent.children if isinstance(t, Tag) and t.name == current.name]
            count = 1 + siblings.index(current)
            path.append(f"{current.name}[{count}]")
            current = current.parent
        return '/' + '/'.join(reversed(path))

    result = {
        "css_selector": generate_css_selector(tag),
        "xpath": generate_xpath(tag),
    }
    return result


def is_text_content_allowed(s):
    if s.endswith("..."):
        return False
    if "...\n" in s:
        return False
    if len(s) < 10:
        return False
    return True


def extract_unique_content(html_pages):
    texts_per_page = []
    target_tags = ['title', 'h1', 'h2', 'h3', 'p', 'span', 'a', 'div']  # ToDo: check if something's missing 

    # Process each page separately
    for html in html_pages:
        soup = BeautifulSoup(html, 'html5lib')
        
        page_texts = []

        for tag in soup.find_all(target_tags):
            text_content = trim_extra_whitespace(tag.get_text(strip=False))
            page_texts.append(text_content)
        
        page_texts = list(set(page_texts))
        texts_per_page.append(page_texts)
    
    all_texts = sum(texts_per_page, [])
    all_texts_counter = Counter(all_texts)
    
    threshold = int(0.7 * len(html_pages))
    
    common_texts = [k for k, v in all_texts_counter.items() if v >= threshold]
    
    unique_texts_per_page = defaultdict(list)
    
    for i, html in enumerate(html_pages):
        soup = BeautifulSoup(html, 'html5lib')
        
        for tag in soup.find_all(target_tags):
            text_content = trim_extra_whitespace(tag.get_text(strip=False))
            if text_content in common_texts:
                continue
            if not is_text_content_allowed(text_content):
                continue
            data = determine_tag_location(tag)
            data["text"] = text_content
            unique_texts_per_page[i].append(data)            
    
    return unique_texts_per_page, all_texts_counter
    


In [77]:
unique_content, all_texts_counter = extract_unique_content(item_html_texts)

In [75]:
example_text

"This dining buffet brings a rustic vibe to urban contemporary design. It’s a modern approach to natural beauty, with an aged black finish defining the piece. And the buffet's color palette is stunningly simple, allowing it to coordinate seamlessly with your existing decor.\n\nProduct Features\nAdjustable\n\nMore About This Product\n\nWeight & Dimensions\n\nDimensions\n\nFull Dimension\n54W x 18D x 40H\n\nWeight\n\nWeight\n59.4lbs."

In [73]:
example_text = unique_content[0][16]["text"]

all_css_selectors = [d["css_selector"] for d in unique_content[0] if d["text"] == example_text]

In [78]:
description_css_selectors = [d["css_selector"] for d in unique_content[0] if "description" in d["css_selector"].lower()]

In [79]:
description_css_selectors

['div.product-description.rte']

In [80]:
soup = BeautifulSoup(item_html_texts[4], 'html5lib')

In [86]:
soup.select(description_css_selectors[0])[0].get_text(strip=True)

'With its sleek lines, gorgeous driftwood tone and cushioned comfort, this 2-piece outdoor loveseat and table set can make your outdoor space feel more like an upscale resort. Made for looks and longevity, this high-quality ensemble is built from eucalyptus wood treated to a deluxe embossed finish to brave the elements. The thick, all-weather seat cushion and back pillow are wrapped in a delightfully neutral Nuvella® fabric that’s a breeze to clean. A pair of throw pillows add a coastal cool touch.More About This ProductWeight & DimensionsDimensionsFull Dimension53W x 30D x 33HWeightWeight87lbs.'

In [88]:
# check out meta "description"

meta_tags = soup.find_all("meta", attrs={"name": "description"})
if meta_tags:
    meta_description = meta_tags[0]["content"]
    print(meta_description)


With its sleek lines, gorgeous driftwood tone and cushioned comfort, this 2-piece outdoor loveseat and table set can make your outdoor space feel more like an upscale resort. Made for looks and longevity, this high-quality ensemble is built from eucalyptus wood treated to a deluxe embossed finish to brave the elements.


In [100]:
winner_candidates = Counter([10, 10, 20, 20, 20, 30, 30, 30])
winner_candidates

Counter({20: 3, 30: 3, 10: 2})

In [102]:
# all first places

max_value = max(winner_candidates.values())
winners = [k for k, v in winner_candidates.items() if v == max_value]
winners

[20, 30]

In [33]:
unique_content[44]

('p',
 'This dining table expertly refines the raw beauty of wood, bringing a rustic vibe to urban contemporary design. It creates an organically beautiful...')

In [36]:
clipboard_copy(data["source"])

In [47]:
qq = [s for s in unique_content if "It’s the dawn of a new day for gray" in s[1]]
qq

[('div',
  '\n  {\n    "settings": {\n      "cart_redirection": false,\n      "layout": "layout--two-col",\n      "thumbnail_position": "below",\n      "gallery_video_autoplay": true,\n      "gallery_video_looping": true,\n      "hover_zoom": "disabled",\n      "click_to_zoom": "always",\n      "money_format": "${{amount}}",\n      "swatches_enable": true\n    },\n    "context": {\n      "product_available": "Add to cart",\n      "product_unavailable": "Sold out"\n    },\n    "product": {"id":8517193957606,"title":"Visola Outdoor Dining Table with 4 Chairs","handle":"visola-outdoor-dining-table-with-4-chairs","description":"It’s the dawn of a new day for gray. Embrace the style trend with this outdoor dining set. Each piece is crafted of HDPE material, combining the exceptional durability and weather resistance you need with the â€œwood lookâ€\x9d you love. The rectangular table and comfortably cushioned chairs take alfresco living to a whole new level. A slatted table top infuses a pl

In [109]:
import html5lib
